In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import rioxarray
import xarray as xr

In [ ]:
tracks = xr.open_dataset(
    './IBTrACS.NA.v04r00.nc'
)

In [ ]:
raft_baseline = xr.open_dataset(
    './01_11vars_RAFT_baseline.nc'
)
raft_ssp245_near_cold = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp245_nearFuture_coldModelMean.nc'
)
raft_ssp245_far_cold = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp245_farFuture_coldModelMean.nc'
)
raft_ssp245_near_hot = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp245_nearFuture_hotModelMean.nc'
)
raft_ssp245_far_hot = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp245_farFuture_hotModelMean.nc'
)
raft_ssp585_near_cold = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp585_nearFuture_coldModelMean.nc'
)
raft_ssp585_far_cold = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp585_farFuture_coldModelMean.nc'
)
raft_ssp585_near_hot = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp585_nearFuture_hotModelMean.nc'
)
raft_ssp585_far_hot = xr.open_dataset(
    './01_11vars_RAFT_TGW_ssp585_farFuture_hotModelMean.nc'
)

In [ ]:
tracks_df = tracks[['name', 'iso_time', 'dist2land']].to_dataframe().reset_index()
tracks_df['time'] = tracks_df.time.astype('datetime64[s]').astype('datetime64[ns]')
tracks_df['over_land'] = (tracks_df.dist2land == 0)

In [ ]:
raft_columns = [
    'stormID', 'datetime_strings', 'vmax_kts',
    'VMPI_t0', 'U200_t0', 'SHRD_t0', 'DELV_6', 'EPTK_t0',
    'LP500_t0', 'PSLV_v3', 'RHLO_t0',
]
raft_df = raft_baseline.to_dataframe().reset_index()[
    raft_columns + ['IBTrACS_vmax']
].rename(columns={
    'IBTrACS_vmax': 'vmax_kts_ibtracs',
}).rename(columns={
    k: f'{k}_baseline' for k in raft_columns[2:]
})
raft_df['datetime_strings'] = pd.to_datetime(raft_df.datetime_strings)
raft_df = raft_df[raft_df.datetime_strings.notna()]

for x, s in [
    (raft_ssp245_near_cold, 'ssp245_near_cold' ),
    (raft_ssp245_far_cold,  'ssp245_far_cold'  ),
    (raft_ssp245_near_hot,  'ssp245_near_hot'  ),
    (raft_ssp245_far_hot,   'ssp245_far_hot'   ),
    (raft_ssp585_near_cold, 'ssp585_near_cold' ),
    (raft_ssp585_far_cold,  'ssp585_far_cold'  ),
    (raft_ssp585_near_hot,  'ssp585_near_hot'  ),
    (raft_ssp585_far_hot,   'ssp585_far_hot'   ),
]:
    df = x.to_dataframe().reset_index()[raft_columns].rename(columns={
        k: f'{k}_{s}' for k in raft_columns[2:]
    })
    df['datetime_strings'] = pd.to_datetime(df.datetime_strings)
    df = df[df.datetime_strings.notna()]
    raft_df = raft_df.merge(
        df,
        how='left',
        on=['stormID', 'datetime_strings'],
    )


In [ ]:
raft_df

In [ ]:
storm_ids = []
storm_names = []
storm_labels = []
storm_years = []
storm_files = []
for storm_id in raft_df.stormID.unique():

    storm = tracks_df[
        (tracks_df.storm==storm_id) &
        tracks_df.time.notna()
    ][['time', 'lat', 'lon', 'name', 'over_land']].reset_index(drop=True).merge(
        raft_df[
            (raft_df.stormID==storm_id)
        ],
        how='left',
        left_on='time',
        right_on='datetime_strings',
    )
    storm_name  = storm.iloc[0]['name'].decode('utf-8').title()
    storm_year  = storm.iloc[0].time.year
    storm_label = f"{storm_name} ({storm_year}) [id:{storm_id}]"
    file_name = f"{storm_name.replace(' ', '_').lower()}_{storm_year}_{storm_id}"
    storm['name'] = storm_name
    storm['year'] = storm_year
    storm['storm_id'] = storm_id
    storm = storm.drop(columns=['datetime_strings', 'stormID'])

    storm_ids.append(storm_id)
    storm_names.append(storm_name)
    storm_labels.append(storm_label)
    storm_years.append(storm_year)
    storm_files.append(file_name)

    schema = pa.Schema.from_pandas(storm, preserve_index=False)
    table = pa.Table.from_pandas(storm, preserve_index=False)
    writer = pa.ipc.new_file(f'../static/data/{file_name}.arrow', schema)
    writer.write(table)
    writer.close()


In [ ]:
storm_reference = pd.DataFrame({
    'id': storm_ids,
    'name': storm_names,
    'year': storm_years,
    'label': storm_labels,
    'value': storm_files,
})

In [ ]:
storm_reference.to_json('../src/lib/storms.js', orient='records', indent=2)